In [ ]:
!pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 12.1 MB/s eta 0:00:00


In [ ]:
# Importar librerías necesarias
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset

# Librerías para el Algoritmo Genético
import random
from deap import base, creator, tools, algorithms

# --- 1. Carga y Preparación de Datos (similar a tu código) ---
# Usamos un transformador para convertir las imágenes a tensores de PyTorch
transform = transforms.ToTensor()

# Descargar y cargar los datasets
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Extraer todos los datos a la memoria para un acceso más rápido durante la HPO
# Tu código original hacía esto al usar un DataLoader con el tamaño completo del dataset.
# Lo mantenemos así para ser consistentes.
x_train = train_dataset.data.view(train_dataset.data.size(0), -1).float() / 255.0
y_train = train_dataset.targets
x_test = test_dataset.data.view(test_dataset.data.size(0), -1).float() / 255.0
y_test = test_dataset.targets

# --- NUEVO: Configuración del dispositivo (GPU o CPU) ---
# Comprueba si hay una GPU disponible (CUDA) y la selecciona. Si no, usa la CPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando el dispositivo: {device}\n")

# Mueve los tensores de datos principales al dispositivo seleccionado una sola vez
x_train = x_train.to(device)
y_train = y_train.to(device)
x_test = x_test.to(device)
y_test = y_test.to(device)

# --- 2. Definición del Modelo (el mismo que tenías) ---
class SoftmaxModel(nn.Module):
    def __init__(self, input_size=784, num_classes=10):
        super(SoftmaxModel, self).__init__()
        self.fc = nn.Linear(input_size, num_classes)

    def forward(self, x):
        return self.fc(x)

# --- 3. Función de Aptitud (Fitness Function) ---
# Esta es la función clave. El algoritmo genético la llamará para evaluar cada "individuo".
# Un individuo representa una combinación de hiperparámetros.
def evaluate_hyperparameters(individual):
    """
    Entrena y evalúa el modelo con un conjunto de hiperparámetros (un individuo).
    Devuelve la precisión en el conjunto de prueba como una tupla, que es la "aptitud".
    """
    # Desempaquetar los hiperparámetros del individuo
    # Gen 0: Tasa de aprendizaje (lr)
    # Gen 1: Tamaño del lote (batch_size)
    # Gen 2: Elección del optimizador (0 para Adam, 1 para SGD)
    lr, batch_size_idx, optimizer_idx = individual

    # Mapear los índices a valores reales
    batch_size_options = [32, 64, 128, 256]
    optimizer_options = ['Adam', 'SGD']

    batch_size = batch_size_options[batch_size_idx]
    optimizer_choice = optimizer_options[optimizer_idx]

    # Crear el modelo y el criterio de pérdida
    # Mueve el modelo al dispositivo seleccionado (GPU o CPU)
    model = SoftmaxModel().to(device)
    criterion = nn.CrossEntropyLoss()

    # Seleccionar el optimizador basado en el gen del individuo
    if optimizer_choice == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=lr)
    else:
        optimizer = optim.SGD(model.parameters(), lr=lr)

    # Crear DataLoader para el entrenamiento con el batch_size del individuo
    train_data = TensorDataset(x_train, y_train)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

    # Entrenamiento (reducimos las épocas para que cada evaluación sea más rápida)
    epochs = 20
    model.train()
    for epoch in range(epochs):
        for data, targets in train_loader:
            # Los datos ya están en el dispositivo correcto porque el TensorDataset se creó con tensores en el dispositivo
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

    # Evaluación
    model.eval()
    with torch.no_grad():
        # x_test ya está en el dispositivo, por lo que no es necesario moverlo de nuevo
        test_outputs = model(x_test)
        _, predicted = torch.max(test_outputs, 1)
        accuracy = (predicted == y_test).float().mean().item()

    # DEAP requiere que la aptitud se devuelva como una tupla
    return (accuracy,)

# --- 4. Configuración del Algoritmo Genético con DEAP ---

# Definir el objetivo: maximizar la precisión (weights=(1.0,))
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
# Definir un individuo como una lista, con el objetivo de fitness definido arriba
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Definir los "genes": cómo se genera cada hiperparámetro aleatoriamente
# Gen 0: Tasa de aprendizaje (un float entre 0.0001 y 0.1)
toolbox.register("attr_lr", random.uniform, 0.0001, 0.1)
# Gen 1: Índice para el tamaño del lote (un entero entre 0 y 3)
toolbox.register("attr_batch_size", random.randint, 0, 3)
# Gen 2: Índice para el optimizador (un entero entre 0 y 1)
toolbox.register("attr_optimizer", random.randint, 0, 1)

# Definir un individuo: una combinación de los 3 genes
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_lr, toolbox.attr_batch_size, toolbox.attr_optimizer), n=1)

# Definir la población: una lista de individuos
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Registrar las operaciones del algoritmo genético
toolbox.register("evaluate", evaluate_hyperparameters)  # Función de aptitud
toolbox.register("mate", tools.cxTwoPoint)            # Cruce
toolbox.register("select", tools.selTournament, tournsize=3) # Selección

# Función de mutación personalizada para manejar diferentes tipos de datos
def mutate_hyperparameters(individual, indpb):
    # Mutar tasa de aprendizaje (float)
    if random.random() < indpb:
        individual[0] = random.uniform(0.0001, 0.1)
    # Mutar tamaño del lote (índice entero)
    if random.random() < indpb:
        individual[1] = random.randint(0, 3)
    # Mutar optimizador (índice entero)
    if random.random() < indpb:
        individual[2] = random.randint(0, 1)
    return individual,

toolbox.register("mutate", mutate_hyperparameters, indpb=0.2) # Probabilidad de mutar cada gen

# --- 5. Ejecución del Algoritmo Genético ---
if __name__ == "__main__":
    # Parámetros del GA
    POP_SIZE = 20  # Tamaño de la población
    CXPB = 0.5     # Probabilidad de cruce
    MUTPB = 0.2    # Probabilidad de mutación
    NGEN = 10      # Número de generaciones

    print("--- Iniciando la optimización de hiperparámetros con Algoritmo Genético ---")

    # Crear la población inicial
    population = toolbox.population(n=POP_SIZE)

    # Guardar el mejor individuo encontrado
    hof = tools.HallOfFame(1)

    # Configurar estadísticas para ver el progreso
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    # Ejecutar el algoritmo
    algorithms.eaSimple(population, toolbox, cxpb=CXPB, mutpb=MUTPB, ngen=NGEN,
                        stats=stats, halloffame=hof, verbose=True)

    # --- 6. Resultados ---
    print("\n--- Optimización Finalizada ---")
    best_individual = hof[0]
    best_params = {
        "learning_rate": best_individual[0],
        "batch_size": [32, 64, 128, 256][best_individual[1]],
        "optimizer": ['Adam', 'SGD'][best_individual[2]]
    }

    print(f"\nMejor individuo encontrado: {best_individual}")
    print(f"Mejores hiperparámetros: {best_params}")
    print(f"Mejor precisión de validación durante la HPO: {best_individual.fitness.values[0]:.4f}")

    # Opcional: Entrenar un modelo final con los mejores hiperparámetros y más épocas
    print("\n--- Entrenando modelo final con los mejores hiperparámetros ---")
    final_model = SoftmaxModel().to(device)
    final_criterion = nn.CrossEntropyLoss()
    if best_params['optimizer'] == 'Adam':
        final_optimizer = optim.Adam(final_model.parameters(), lr=best_params['learning_rate'])
    else:
        final_optimizer = optim.SGD(final_model.parameters(), lr=best_params['learning_rate'])

    final_train_data = TensorDataset(x_train, y_train)
    final_train_loader = DataLoader(final_train_data, batch_size=best_params['batch_size'], shuffle=True)

    final_epochs = 100 # Entrenamos por más tiempo con los mejores parámetros
    final_model.train()
    for epoch in range(final_epochs):
        for data, targets in final_train_loader:
            # Los datos ya están en el dispositivo correcto
            final_optimizer.zero_grad()
            outputs = final_model(data)
            loss = final_criterion(outputs, targets)
            loss.backward()
            final_optimizer.step()
        if (epoch + 1) % 10 == 0:
            print(f"Época final [{epoch+1}/{final_epochs}], Pérdida: {loss.item():.4f}")

    # Evaluación final
    final_model.eval()
    with torch.no_grad():
        final_outputs = final_model(x_test)
        _, final_predicted = torch.max(final_outputs, 1)
        final_accuracy = (final_predicted == y_test).float().mean().item()

    print(f"\nPrecisión final en el conjunto de prueba: {final_accuracy:.4f}")



100%|██████████| 9.91M/9.91M [00:01<00:00, 5.12MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 133kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.27MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.10MB/s]


Usando el dispositivo: cuda

--- Iniciando la optimización de hiperparámetros con Algoritmo Genético ---
gen	nevals	avg    	std      	min   	max   
0  	20    	0.91591	0.0107891	0.8851	0.9246
1  	11    	0.921165	0.00546895	0.9042	0.926 
2  	15    	0.924215	0.000898479	0.9217	0.9258
3  	11    	0.92477 	0.000689277	0.9232	0.9258
4  	14    	0.924325	0.00212105 	0.9167	0.9265
5  	11    	0.924715	0.0023176  	0.9153	0.9267
6  	12    	0.92469 	0.00194728 	0.9177	0.9268
7  	14    	0.924565	0.00176274 	0.9189	0.9271
8  	12    	0.92436 	0.00283221 	0.9128	0.9268
9  	13    	0.9249  	0.00107331 	0.9221	0.9263
10 	11    	0.923915	0.00574642 	0.8992	0.9263

--- Optimización Finalizada ---

Mejor individuo encontrado: [0.08392657225847973, 0, 1]
Mejores hiperparámetros: {'learning_rate': 0.08392657225847973, 'batch_size': 32, 'optimizer': 'SGD'}
Mejor precisión de validación durante la HPO: 0.9267

--- Entrenando modelo final con los mejores hiperparámetros ---
Época final [10/100], Pérdida: 0.1467
Ép